In [2]:
from google.colab import drive
import os
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')
# Load DataFrame from the CSV file
csv_file_path = '/content/drive/My Drive/Projet 2CS/DATASET_IF/dataset_if.csv'
df = pd.read_csv(csv_file_path)

# Now df contains the DataFrame with the data from the CSV file
print("DataFrame loaded successfully from:", csv_file_path)


Mounted at /content/drive
DataFrame loaded successfully from: /content/drive/My Drive/Projet 2CS/DATASET_IF/dataset_if.csv


In [7]:
!pip install deap
!pip install astor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.8 MB/s eta 0:00:00


In [8]:
import ast
import astor

def replace_expression_condition(code, line_number, old_expression, new_expression):
    # Parse the code into an abstract syntax tree (AST)
    print("Old Expression:", old_expression)
    print("New Expression:", new_expression)
    tree = ast.parse(code)

    # Define a visitor to traverse the AST and perform replacements
    class ReplaceExpression(ast.NodeTransformer):
        def visit_If(self, node):
            # Check if the node corresponds to the specified line number
            if getattr(node, 'lineno', None) == line_number:
                # Try parsing the old and new expressions into AST nodes
                try:
                    old_expr_ast = ast.parse(old_expression, mode='eval').body
                    new_expr_ast = ast.parse(new_expression, mode='eval').body
                except SyntaxError as e:
                    print("SyntaxError:", e)
                    return node

                # Compare the old expression in the AST with the given old expression
                if self.compare_expr(node.test, old_expr_ast):
                    # Replace the old expression with the new expression
                    node.test = new_expr_ast

            # Continue to traverse the child nodes
            self.generic_visit(node)
            return node

        def compare_expr(self, expr1, expr2):
            # Compare the string representations of the AST nodes
            return ast.dump(expr1) == ast.dump(expr2)

    # Instantiate the visitor and traverse the AST
    transformer = ReplaceExpression()
    transformed_tree = transformer.visit(tree)

    # Generate Python code from the modified AST
    modified_code = astor.to_source(transformed_tree)

    return modified_code


In [9]:
import subprocess
import re

def evaluate_program(erroneous_program):
    # Execute the erroneous program as a subprocess
    result = subprocess.run(['python', '-c', erroneous_program], capture_output=True, text=True)

    # Initialize counters
    total_tests = 0
    total_failed_tests = 0

    # Extract information from the stderr
    stderr_output = result.stderr

    # Use regex to find the number of tests ran
    match = re.search(r'Ran (\d+) tests', stderr_output)
    if match:
        total_tests = int(match.group(1))

    # Use regex to find the number of failures and errors
    match_failures = re.search(r'FAILED \((failures=(\d+))?(, )?(errors=(\d+))?\)', stderr_output)
    if match_failures:
        failures = match_failures.group(2)
        errors = match_failures.group(5)
        if failures:
            total_failed_tests += int(failures)
        if errors:
            total_failed_tests += int(errors)

    # Calculate the number of successful tests
    successful_tests = total_tests - total_failed_tests
    failed_tests = total_failed_tests

    print('total_tests : ', total_tests)
    print('failed_tests (including errors) : ', total_failed_tests)
    print('successful_tests : ', successful_tests)

    return total_tests, failed_tests, successful_tests

In [25]:
# Assuming df is your DataFrame containing the dataset
erroneous_program = df.iloc[66]['Content']
print(erroneous_program)

lines = erroneous_program.split('\n')
for i, line in enumerate(lines, start=1):
    print(f"Line {i}: {line}")

import unittest
class AbsMinusWrongIf2:
    @staticmethod
    def absM(i, j):
        result = 0
        k = 0
        if i <= j:
            k = k + 1
        if k == 0 and i != j:  # (k == 1 and i != j)
            result = j - i
        else:
            result = i - j
        return result
    
# Define a test class inheriting from unittest.TestCase
class TestAddFunction(unittest.TestCase):
    def test_0(self):
        self.assertEqual(AbsMinusWrongIf2.absM(1, 2), 1)  

    def test_1(self):
        self.assertEqual(AbsMinusWrongIf2.absM(5, 6), 1)  

    def test_2(self):
        self.assertEqual(AbsMinusWrongIf2.absM(5, 8), 3)  

    def test_3(self):
        self.assertEqual(AbsMinusWrongIf2.absM(1, 6), 5)  
        
    def test_4(self):
        self.assertEqual(AbsMinusWrongIf2.absM(1, 8), 7)  
    
    def test_5(self):
        self.assertEqual(AbsMinusWrongIf2.absM(10, 2), 8)  
    
    def test_6(self):
        self.assertEqual(AbsMinusWrongIf2.absM(10, 6), 4)  
        
  

In [10]:
from pyparsing import Forward, Literal, Word, alphas, nums, alphanums

# Define the grammar for the prefix expression
expr = Forward()
identifier = Word(alphas + '_', alphanums + '_')
operand = Word(nums) | (Literal('-').suppress() + Word(nums))
op = Literal('+') | Literal('-') | Literal('*') | Literal('/') | Literal('<=') | Literal('>=') | Literal('<') | Literal('==') | Literal('!=') | Literal('>') | Literal('and') | Literal('or')
open_paren = Literal("(").suppress()
close_paren = Literal(")").suppress()
comma = Literal(",").suppress()
expr <<= op + open_paren + expr + comma + expr + close_paren | identifier | operand

# Define the infix notation with the correct operator precedence
def infix_action(tokens):
    if len(tokens) == 1:
        return tokens[0]
    elif len(tokens) == 2:  # Unary operators like '-' or 'not'
        return f"{tokens[0]}({tokens[1]})"
    else:
        return f"({tokens[1]} {tokens[0]} {tokens[2]})"

expr.setParseAction(infix_action)
# Example usage:
prefix_expr = "and(<=(mm1234, var1), or(==(var2, xyz), >(var3, var4)))"
infix_expr = expr.parseString(prefix_expr)[0]
print("Infix expression:", infix_expr)

Infix expression: ((mm1234 <= var1) and ((var2 == xyz) or (var3 > var4)))


In [27]:
import ast
import keyword
import builtins

def extract_variables_constants(erroneous_program):
    tree = ast.parse(erroneous_program)
    variables = set()
    constants = set()

    # Collecter les noms des fonctions pour les exclure des variables
    function_names = {node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)}

    # Collecter les noms définis au niveau du module (par exemple, modules importés, classes)
    module_level_names = set()
    for node in ast.walk(tree):
        if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
            module_level_names.add(node.name)
        elif isinstance(node, (ast.Import, ast.ImportFrom)):
            for alias in node.names:
                module_level_names.add(alias.name)

    # Ajouter les noms des fonctions intégrées pour les exclure des variables
    builtins_names = set(dir(builtins))

    # Collecter les noms définis dans les méthodes de test
    test_method_names = set()
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name.startswith("test_"):
            # Exclure les variables définies dans les méthodes de test
            for child_node in ast.walk(node):
                if isinstance(child_node, ast.Name) and not isinstance(child_node.ctx, ast.Store):
                    test_method_names.add(child_node.id)

    # Parcourir l'arbre syntaxique pour extraire les noms des variables et constantes
    for node in ast.walk(tree):
        if isinstance(node, ast.Name) and not isinstance(node.ctx, ast.Store):
            # Vérifier si le nom n'est pas un mot-clé, une fonction, un intégré, un nom de niveau module, ou un nom de méthode de test
            if node.id not in keyword.kwlist and node.id not in function_names \
               and node.id not in module_level_names and node.id not in builtins_names \
               and node.id not in test_method_names:
                variables.add(node.id)
        elif isinstance(node, ast.Constant) and node.value != '__main__':  # Pour Python 3.8+
            if isinstance(node.value, (int, float, str)):
                constants.add(node.value)
        elif isinstance(node, ast.Num):  # Pour compatibilité avec Python 3.7 et versions antérieures
            constants.add(node.n)

    return list(variables), list(constants)

variables, constants = extract_variables_constants(erroneous_program)
print("Variables used in the erroneous program:", variables)
print("Constants used in the erroneous program:", constants)

Variables used in the erroneous program: ['i', 'j', 'result', 'k']
Constants used in the erroneous program: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10]


In [30]:
import random
import operator
import numpy as np
import threading
import queue
from deap import base, creator, tools, gp, algorithms
import subprocess
import re

class Bool(object):
  TRUE = True
  FALSE = False

class Int(int):
    def __bool__(self):
        return False

    def __new__(cls, value):
        if isinstance(value, bool):
            raise ValueError("Cannot create an Int instance from a bool")
        return super().__new__(cls, value)

    def __repr__(self):
        return f"Int({int(self)})"

# Define the primitive set for the symbolic regression problem
pset = gp.PrimitiveSetTyped("MAIN", [Int, Int], Bool)  # Set arity equal to the number of variables

# Add comparison operators which take two integers and return a boolean
pset.addPrimitive(operator.lt, [Int, Int], Bool)
pset.addPrimitive(operator.le, [Int, Int], Bool)
pset.addPrimitive(operator.eq, [Int, Int], Bool)
pset.addPrimitive(operator.ne, [Int, Int], Bool)
pset.addPrimitive(operator.gt, [Int, Int], Bool)
pset.addPrimitive(operator.ge, [Int, Int], Bool)
pset.addPrimitive(operator.add, [Int, Int],Int)  # Addition (+)
pset.addPrimitive(operator.sub, [Int, Int],Int)  # Subtraction (-)
pset.addPrimitive(operator.mul, [Int, Int],Int)

# Add logical operators which take two booleans and return a boolean
pset.addPrimitive(operator.and_, [Bool, Bool], Bool)
pset.addPrimitive(operator.or_, [Bool, Bool], Bool)
# Add terminals: True and False of type bool, and ephemeral constants of type int
pset.addTerminal(True,Bool)
pset.addTerminal(False,Bool)
pset.addEphemeralConstant("rand101", lambda: random.randint(0, 1), Int)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def translate_expr(individual, variables):
    expr = str(individual)
    # Replace the primitive set names with mathematical operators
    expr = expr.replace("eq", "==") # Equal to
    expr = expr.replace("add", "+")
    expr = expr.replace("sub", "-")
    expr = expr.replace("mul", "*")
    expr = expr.replace("lt", "<")  # Less than
    expr = expr.replace("le", "<=") # Less than or equal to
    expr = expr.replace("ne", "!=") # Not equal to
    expr = expr.replace("gt", ">")  # Greater than
    expr = expr.replace("ge", ">=")
    expr = expr.replace("and_", "and")  # Logical AND
    expr = expr.replace("or_", "or")
    # Replace ephemeral constants with their values
    expr = expr.replace("rand101", str(random.randint(-10, 10)))
    # Replace ARG0, ARG1, ..., ARGn with the variable names
    for i, var in enumerate(variables):
        arg_name = "ARG{}".format(i)
        # Replace the argument name with the variable name
        expr = expr.replace(arg_name, var)
    return expr

def get_variable_name(node, variables):
    """
    Helper function to get the variable name from a node, if it exists in the given variables list.
    """
    if isinstance(node, ast.Name):
        if node.id in variables:
            return node.id
    return None

def has_duplicate_vars(expr, variables):
    """
    Check if an expression contains duplicate variables in a direct comparison.
    """
    try:
        tree = ast.parse(expr, mode='eval')
        for node in ast.walk(tree):
            if isinstance(node, ast.Compare):
                left = get_variable_name(node.left, variables)
                for comparator in node.comparators:
                    right = get_variable_name(comparator, variables)
                    if left and left == right:
                        return True
    except (SyntaxError, ValueError):
        pass
    return False

def eval_individual(individual, variables, result_queue, timeout_event):
    try:
        new_expression = translate_expr(individual, variables)
        new_expression = expr.parseString(str(new_expression))
        erroneous_code = replace_expression_condition(erroneous_program, 9 , "k == 0 and i != j", str(new_expression)[2:-2].strip())
        # print(erroneous_code)
        total_tests, failed_tests, successful_tests = evaluate_program(erroneous_code)
        if has_duplicate_vars(str(new_expression)[2:-2].strip(), variables):
          # Apply a penalty to the fitness score
          failed_tests += 10
        result_queue.put((failed_tests, total_tests))
    except Exception as e:
        print(f"Evaluation error: {e}")
        result_queue.put((float('25'), 0))
    finally:
        timeout_event.set()  # Set the event to signal the end of evaluation

def evalSymbReg(individual, variables, timeout=1):
    timeout_event = threading.Event()  # Event to signal timeout
    result_queue = queue.Queue()  # Queue to store evaluation result

    # Start a new thread to execute eval_individual
    eval_thread = threading.Thread(target=eval_individual, args=(individual, variables, result_queue, timeout_event))
    eval_thread.start()

    # Wait for the thread to finish or timeout
    eval_thread.join(timeout=timeout)  # Timeout set to the given seconds

    if not timeout_event.is_set():
        print("Evaluation timed out.")
        return float('25'), 0  # Return infinity and 0 if evaluation timed out

    evaluation_result = result_queue.get()  # Get the result from the queue
    if evaluation_result is None:
        print("Evaluation failed.")
        return float('25'), 0  # Return infinity and 0 if evaluation failed

    failed_tests, total_tests = evaluation_result
    return failed_tests,

def evolution(toolbox, pop, hof, stats, ngen):
    log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen,
                              stats=stats, halloffame=hof, verbose=True)
    best_individual = hof[0]
    best_fitness = best_individual.fitness.values[0]

    for gen, record in enumerate(log[1:]):
        for i in range(len(record)):
            print([record[i]['gen'], record[i]['nevals'], record[i]['avg'], record[i]['std'], record[i]['min'], record[i]['max'], best_individual])

    best_gen = record[-1]['gen'] if record else None

    return float(best_gen) if best_gen is not None else None


def main():
    random.seed(42)
    if len(variables) < 1:
        print("Erreur: Nombre incorrect de variables extraites.")
        return

    variables_list = list(variables)  # Convert the set to a list
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    toolbox.register("evaluate", evalSymbReg, variables=variables_list)  # Pass the list of variables
    toolbox.register("select", tools.selTournament, tournsize=3)  # Register tournament selection operator
    toolbox.register("mate", gp.cxOnePoint)  # Register crossover operator
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)  # Register mutation operator
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)  # Register mutation operator

    ngen = 0
    # Run the evolution process
    best_gen = evolution(toolbox, pop, hof, stats, ngen)
    print(f"Best generation: {best_gen}")

    if best_gen != -1:
        # Print the best individual
        best_individual = hof[0]
        best_expr_str = translate_expr(best_individual, variables_list)  # Pass the list of variables
        best_expr_str2 = expr.parseString(str(best_expr_str))
        print("Best individual infix expression:", best_expr_str2)
    else:
        print("No solution found within the time limit.")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/deap/gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Old Expression: k == 0 and i != j
New Expression: (i > i)
total_tests :  9
failed_tests (including errors) :  5
successful_tests :  4
Old Expression: k == 0 and i != j
New Expression: (1 <= i)
total_tests :  9
failed_tests (including errors) :  4
successful_tests :  5
Old Expression: k == 0 and i != j
New Expression: (i != 0)
total_tests :  9
failed_tests (including errors) :  4
successful_tests :  5
Old Expression: k == 0 and i != j
New Expression: ((i - i) != (j - i))
total_tests :  9
failed_tests (including errors) :  4
successful_tests :  5
Old Expression: k == 0 and i != j
New Expression: (i <= (1 - i))
total_tests :  9
failed_tests (including errors) :  5
successful_tests :  4
Old Expression: k == 0 and i != j
New Expression: (True and False)
total_tests :  9
failed_tests (including errors) :  5
successful_tests :  4
Old Expression: k == 0 and i != j
New Expression: (i <= 0)
total_tests :  9
failed_tests (including errors) :  5
successful_tests :  4
Old Expression: k == 0 and i !